In [1]:
import pandas as pd
import yaml
import requests
import json

In [2]:
secrets_location = "P:\\My Documents\\Github\\Supporting Files\\secrets.yml"
secrets = open(secrets_location)
secrets = yaml.load(secrets, Loader=yaml.FullLoader)['SharePoint_LGBF']
SP_clientsecret = secrets['client_secret']
SP_tenantid = secrets['tenant_id']
SP_tenant = "stirlingcounciluk"
SP_clientid = secrets['client_id'] + '@' + SP_tenantid


In [3]:
# Establish data to be passed as part of POST request to obtain session token from SharePoint
data = {
    'grant_type': 'client_credentials',
    'resource': "00000003-0000-0ff1-ce00-000000000000/" + SP_tenant + ".sharepoint.com@" + SP_tenantid,
    'client_id': SP_clientid,
    'client_secret': SP_clientsecret,
}

# Establish headers to be passed as part of POST request to obtain session token from SharePoint
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Construct url variable for token request
url = "https://accounts.accesscontrol.windows.net/" + \
    SP_tenantid + "/tokens/OAuth/2"

# Send POST request to obtain session token and then load the response into json_data
r = requests.post(url, data=data, headers=headers)
json_data = json.loads(r.text)


In [4]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept': 'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + \
    ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Alternate Line Types')/items"

Items = []

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.
def getItems(url):
    Itemsrequest = requests.get(url, headers=headers)
    Itemsjson = Itemsrequest.json()
    Item = pd.json_normalize(Itemsjson['d']['results'])
    Items.append(Item)
    if '__next' in Itemsjson['d']:
        newurl = Itemsjson['d']['__next']
        getItems(newurl)


getItems(url)

alternateLines = pd.DataFrame()
for i in Items:
    alternateLines = alternateLines.append(i)

alternateLines


,FileSystemObjectType,Id,ServerRedirectedEmbedUri,ServerRedirectedEmbedUrl,ID,ContentTypeId,Title,Modified,Created,AuthorId,...,FieldValuesAsText.__deferred.uri,FieldValuesForEdit.__deferred.uri,File.__deferred.uri,Folder.__deferred.uri,LikedByInformation.__deferred.uri,ParentList.__deferred.uri,Properties.__deferred.uri,Versions.__deferred.uri,IndicatorId.__metadata.type,IndicatorId.results
0,0,1,None,,1,0x0100A3C8811D84B3D24FA26C040BA00BBD9500892711...,None,2022-08-19T10:04:13Z,2021-03-30T09:40:37Z,10,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,Collection(Edm.Int32),"[79, 80, 81, 82, 85, 86, 87, 88, 89, 90]"


In [5]:
# Drop system columns not needed in dataframe
alternateLines = alternateLines.drop(columns=['FileSystemObjectType', 'Id', 'ServerRedirectedEmbedUri', 'ServerRedirectedEmbedUrl', 'GetDlpPolicyTip.__deferred.uri', 'FieldValuesAsHtml.__deferred.uri', 'FieldValuesAsText.__deferred.uri', 'FieldValuesForEdit.__deferred.uri', 'File.__deferred.uri', 'Folder.__deferred.uri', 'LikedByInformation.__deferred.uri', 'ParentList.__deferred.uri', 'Properties.__deferred.uri',
                                                 'Versions.__deferred.uri', 'Attachments', 'GUID', '__metadata.id', '__metadata.uri', '__metadata.etag', '__metadata.type', 'FirstUniqueAncestorSecurableObject.__deferred.uri', 'RoleAssignments.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'ContentType.__deferred.uri', 'ID', 'Modified', 'Created', 'AuthorId', 'EditorId', 'OData__UIVersionString', 'ContentTypeId', 'ComplianceAssetId','IndicatorId.__metadata.type','Title'])

# Rename columns appropriately
alternateLines = alternateLines.rename(columns={'PeriodsAffected_x0028_semicolond': 'Period','AlternateLineType_x0028_IfApplic' : 'LineType','IndicatorId.results': 'IndicatorID'})
alternateLines


,Period,LineType,IndicatorID
0,2019-20;2020-21,Dashed,"[79, 80, 81, 82, 85, 86, 87, 88, 89, 90]"


In [6]:
alternateLines = alternateLines.assign(Period=alternateLines.Period.str.split(';'))
alternateLines = alternateLines.explode('Period')
alternateLines = alternateLines.explode('IndicatorID')
alternateLines


,Period,LineType,IndicatorID
0,2019-20,Dashed,79
0,2019-20,Dashed,80
0,2019-20,Dashed,81
0,2019-20,Dashed,82
0,2019-20,Dashed,85
0,2019-20,Dashed,86
0,2019-20,Dashed,87
0,2019-20,Dashed,88
0,2019-20,Dashed,89
0,2019-20,Dashed,90


In [7]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept': 'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + \
    ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Indicator%20Information')/items"

Items = []

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.


def getItems(url):
    Itemsrequest = requests.get(url, headers=headers)
    Itemsjson = Itemsrequest.json()
    Item = pd.json_normalize(Itemsjson['d']['results'])
    Items.append(Item)
    if '__next' in Itemsjson['d']:
        newurl = Itemsjson['d']['__next']
        getItems(newurl)


getItems(url)

indicatorInformation = pd.DataFrame()
for i in Items:
    indicatorInformation = indicatorInformation.append(i)

indicatorInformation = indicatorInformation[['Id','Code_x0028_SortableA_x002d_Z_x00']]
indicatorInformation


,Id,Code_x0028_SortableA_x002d_Z_x00
0,18,ENV 01a
1,19,ENV 02a
2,20,ENV 06
3,21,ENV 07a
4,22,ENV 03a
...,...,...
97,116,HSN 05
98,117,CHN 24
99,118,ECON 11
0,119,ECON 12a


In [8]:
alternateLines = alternateLines.merge(indicatorInformation, left_on = ['IndicatorID'], right_on=['Id'])
alternateLines = alternateLines[['Code_x0028_SortableA_x002d_Z_x00','Period','LineType']]
alternateLines = alternateLines.rename(columns={'Code_x0028_SortableA_x002d_Z_x00': 'Code'})
alternateLines['Key_CodePeriod'] = alternateLines['Code'] + alternateLines['Period']
alternateLines


,Code,Period,LineType,Key_CodePeriod
0,CHN 04,2019-20,Dashed,CHN 042019-20
1,CHN 04,2020-21,Dashed,CHN 042020-21
2,CHN 05,2019-20,Dashed,CHN 052019-20
3,CHN 05,2020-21,Dashed,CHN 052020-21
4,CHN 06,2019-20,Dashed,CHN 062019-20
5,CHN 06,2020-21,Dashed,CHN 062020-21
6,CHN 07,2019-20,Dashed,CHN 072019-20
7,CHN 07,2020-21,Dashed,CHN 072020-21
8,CHN 12a,2019-20,Dashed,CHN 12a2019-20
9,CHN 12a,2020-21,Dashed,CHN 12a2020-21


In [9]:
alternateLines.to_csv("Data Files//Alternative Lines.csv", index=False, encoding='utf-8-sig')